In [2]:
# install.packages("markdown")
library(knitr)


# Loading
library(dplyr)

# to get R2 in lmer
# library(sjstats)
# library(sjPlot)


# fit and test linear mixed models
library(lme4)
library(lmerTest)

# contrasts
# install.packages("pbkrtest")
library(emmeans)

# function pvalue is here
library(scales)

# code to add *, ** and *** for significances
makeStars <- function(x){
  stars <- c("***", "**", "*", "")
  vec <- c(0, 0.001, 0.01, 0.05, 1.1)
  i <- findInterval(x, vec)
  stars[i]
}

# xlsx files
df <- read.csv("data/main.csv")

df <- df[df['engine']=='semantic',]

# Factorization

In [3]:
# factorize topics
df$region <- as.factor(df$region)
df$browser <- as.factor(df$browser)
df$id <- as.factor(df$id)

df$topic <- as.factor(df$topic)
df$trt <- factor(df$trt, levels = c('risks', 'benefits'))


# Full Analisys

In [4]:
options(width = 10000)

fit <- lmer(valence ~   trt * topic + region + browser + (day|id),  data = df)

print(anova(fit, type='II'))
print(summary(fit))


boundary (singular) fit: see help('isSingular')



Type II Analysis of Variance Table with Satterthwaite's method
          Sum Sq Mean Sq NumDF DenDF   F value Pr(>F)    
trt       453.33  453.33     1 14335 1649.8787 <2e-16 ***
topic     686.82  137.36     5 14335  499.9312 <2e-16 ***
region      0.00    0.00     1 14335    0.0024 0.9610    
browser     0.00    0.00     1 14335    0.0003 0.9864    
trt:topic 668.97  133.79     5 14335  486.9420 <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
Linear mixed model fit by REML. t-tests use Satterthwaite's method ['lmerModLmerTest']
Formula: valence ~ trt * topic + region + browser + (day | id)
   Data: df

REML criterion at convergence: 22263.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.3322 -0.6362 -0.3818  1.0728  2.5442 

Random effects:
 Groups   Name        Variance  Std.Dev.  Corr
 id       (Intercept) 0.000e+00 0.000e+00     
          day         1.612e-15 4.015e-08  NaN
 Residual             2.748e-01 5.242e-01     
Number of obs


Correlation matrix not shown by default, as p = 14 > 12.
Use print(summary(fit), correlation=TRUE)  or
    vcov(summary(fit))        if you need it




optimizer (nloptwrap) convergence code: 0 (OK)
boundary (singular) fit: see help('isSingular')



# Contrasts for Treatment: Risks vs Benefits

In [5]:
# asymptotic is used for approximations because other methods ("kenward-roger", "satterthwaite") are 
# computationally too expensive
# https://link.springer.com/article/10.3758/s13428-016-0809-y


the_means <- emmeans(fit, ~  trt |(topic), lmer.df = "asymptotic")
contrast_trt <- pairs(the_means)
# print(contrast_trt)


# create dataframe of contrasts
em <- pairs(the_means, interaction = "pairwise", infer = c(TRUE, TRUE)) %>%  rbind() 

# bonferroni method for 16 tests 
em

 topic            trt_pairwise     estimate     SE  df asymp.LCL asymp.UCL z.ratio p.value
 coffee           risks - benefits  -0.1111 0.0229 Inf    -0.171   -0.0508  -4.859  <.0001
 covid vaccines   risks - benefits   0.3333 0.0213 Inf     0.277    0.3895  15.658  <.0001
 cryptocurrencies risks - benefits  -0.8675 0.0213 Inf    -0.924   -0.8114 -40.753  <.0001
 internet         risks - benefits  -0.6000 0.0207 Inf    -0.655   -0.5453 -28.958  <.0001
 social media     risks - benefits  -0.7708 0.0213 Inf    -0.827   -0.7146 -36.135  <.0001
 vaccines         risks - benefits  -0.0703 0.0215 Inf    -0.127   -0.0136  -3.273  0.0064

Results are averaged over some or all of the levels of: region, browser 
Degrees-of-freedom method: asymptotic 
Confidence level used: 0.95 
Conf-level adjustment: bonferroni method for 6 estimates 
P value adjustment: bonferroni method for 6 tests 

In [6]:
print (the_means %>%  rbind() %>% as.data.frame() 
        %>% mutate(across(where(is.numeric), round, 4)))
     

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `across(where(is.numeric), round, 4)`.
Caused by warning:
! The `...` argument of `across()` is deprecated as of dplyr 1.1.0.
Supply arguments directly to `.fns` through an anonymous function instead.

  # Previously
  across(a:b, mean, na.rm = TRUE)

  # Now
  across(a:b, \(x) mean(x, na.rm = TRUE))”


              topic      trt  emmean     SE  df asymp.LCL asymp.UCL
1            coffee    risks  0.2222 0.0165 Inf    0.1749    0.2695
2            coffee benefits  0.3333 0.0158 Inf    0.2880    0.3787
3    covid vaccines    risks  0.3333 0.0154 Inf    0.2891    0.3776
4    covid vaccines benefits  0.0000 0.0147 Inf   -0.0420    0.0420
5  cryptocurrencies    risks -0.6675 0.0154 Inf   -0.7118   -0.6233
6  cryptocurrencies benefits  0.2000 0.0147 Inf    0.1580    0.2420
7          internet    risks -0.5000 0.0147 Inf   -0.5420   -0.4580
8          internet benefits  0.1000 0.0147 Inf    0.0580    0.1420
9      social media    risks -0.3333 0.0155 Inf   -0.3778   -0.2889
10     social media benefits  0.4375 0.0147 Inf    0.3955    0.4795
11         vaccines    risks  0.3297 0.0157 Inf    0.2848    0.3747
12         vaccines benefits  0.4000 0.0147 Inf    0.3580    0.4420


In [7]:
# convert to data frame and sort
emdf <-  as.data.frame(em) %>% arrange(desc(topic))

# add significative starts
emdf['sig'] <- makeStars(emdf$p.value)

# restrict p-values decimals and add <0.0001 where correspond
emdf[,"p.value"] <- pvalue(emdf[,"p.value"], accuracy = 0.001)

# round all values to 4 decimals
embf <- emdf %>% mutate(across(where(is.numeric), round, 4))

print(embf)
     


             topic     trt_pairwise estimate     SE  df asymp.LCL asymp.UCL  z.ratio p.value sig
1         vaccines risks - benefits  -0.0703 0.0215 Inf   -0.1269   -0.0136  -3.2726   0.006  **
2     social media risks - benefits  -0.7708 0.0213 Inf   -0.8271   -0.7146 -36.1353  <0.001 ***
3         internet risks - benefits  -0.6000 0.0207 Inf   -0.6547   -0.5453 -28.9575  <0.001 ***
4 cryptocurrencies risks - benefits  -0.8675 0.0213 Inf   -0.9237   -0.8114 -40.7527  <0.001 ***
5   covid vaccines risks - benefits   0.3333 0.0213 Inf    0.2772    0.3895  15.6584  <0.001 ***
6           coffee risks - benefits  -0.1111 0.0229 Inf   -0.1714   -0.0508  -4.8593  <0.001 ***


# Contrasts for Treatment: Health vs Technology

In [8]:
# asymptotic is used for approximations because other methods ("kenward-roger", "satterthwaite") are 
# computationally too expensive
# https://link.springer.com/article/10.3758/s13428-016-0809-y


the_means <- emmeans(fit, ~  topic |(trt), lmer.df = "asymptotic")
contrast_trt <- pairs(the_means)
# print(contrast_trt)


# create dataframe of contrasts
em <- pairs(the_means, interaction = "pairwise", infer = c(TRUE, TRUE)) %>%  rbind() 

# bonferroni method for 16 tests 
em

 trt      topic_pairwise                    estimate     SE  df asymp.LCL asymp.UCL z.ratio p.value
 risks    coffee - covid vaccines           -0.11111 0.0226 Inf   -0.1822  -0.04003  -4.915  <.0001
 risks    coffee - cryptocurrencies          0.88976 0.0226 Inf    0.8187   0.96083  39.357  <.0001
 risks    coffee - internet                  0.72222 0.0221 Inf    0.6528   0.79162  32.719  <.0001
 risks    coffee - social media              0.55556 0.0226 Inf    0.4843   0.62676  24.529  <.0001
 risks    coffee - vaccines                 -0.10752 0.0228 Inf   -0.1791  -0.03591  -4.721  0.0001
 risks    covid vaccines - cryptocurrencies  1.00087 0.0218 Inf    0.9322   1.06954  45.826  <.0001
 risks    covid vaccines - internet          0.83333 0.0213 Inf    0.7664   0.90026  39.146  <.0001
 risks    covid vaccines - social media      0.66667 0.0219 Inf    0.5979   0.73547  30.464  <.0001
 risks    covid vaccines - vaccines          0.00359 0.0220 Inf   -0.0656   0.07281   0.163  1.0000


In [9]:
print (the_means %>%  rbind() %>% as.data.frame() 
         %>% mutate(across(where(is.numeric), round, 4)))
     

        trt            topic  emmean     SE  df asymp.LCL asymp.UCL
1     risks           coffee  0.2222 0.0165 Inf    0.1749    0.2695
2     risks   covid vaccines  0.3333 0.0154 Inf    0.2891    0.3776
3     risks cryptocurrencies -0.6675 0.0154 Inf   -0.7118   -0.6233
4     risks         internet -0.5000 0.0147 Inf   -0.5420   -0.4580
5     risks     social media -0.3333 0.0155 Inf   -0.3778   -0.2889
6     risks         vaccines  0.3297 0.0157 Inf    0.2848    0.3747
7  benefits           coffee  0.3333 0.0158 Inf    0.2880    0.3787
8  benefits   covid vaccines  0.0000 0.0147 Inf   -0.0420    0.0420
9  benefits cryptocurrencies  0.2000 0.0147 Inf    0.1580    0.2420
10 benefits         internet  0.1000 0.0147 Inf    0.0580    0.1420
11 benefits     social media  0.4375 0.0147 Inf    0.3955    0.4795
12 benefits         vaccines  0.4000 0.0147 Inf    0.3580    0.4420


In [10]:
# convert to data frame and sort
emdf <-  as.data.frame(em) %>% arrange(trt)

# add significative starts
emdf['sig'] <- makeStars(emdf$p.value)

# restrict p-values decimals and add <0.0001 where correspond
emdf[,"p.value"] <- pvalue(emdf[,"p.value"], accuracy = 0.001)

# round all values to 4 decimals
embf <- emdf %>% mutate(across(where(is.numeric), round, 4))

print(embf)

        trt                    topic_pairwise estimate     SE  df asymp.LCL asymp.UCL  z.ratio p.value sig
1  benefits           coffee - covid vaccines   0.3333 0.0216 Inf    0.2655    0.4011  15.4595  <0.001 ***
2  benefits         coffee - cryptocurrencies   0.1333 0.0216 Inf    0.0655    0.2011   6.1838  <0.001 ***
3  benefits                 coffee - internet   0.2333 0.0216 Inf    0.1655    0.3011  10.8216  <0.001 ***
4  benefits             coffee - social media  -0.1042 0.0216 Inf   -0.1720   -0.0364  -4.8312  <0.001 ***
5  benefits                 coffee - vaccines  -0.0667 0.0216 Inf   -0.1345    0.0011  -3.0920   0.060    
6  benefits covid vaccines - cryptocurrencies  -0.2000 0.0207 Inf   -0.2651   -0.1349  -9.6525  <0.001 ***
7  benefits         covid vaccines - internet  -0.1000 0.0207 Inf   -0.1651   -0.0349  -4.8263  <0.001 ***
8  benefits     covid vaccines - social media  -0.4375 0.0207 Inf   -0.5026   -0.3724 -21.1149  <0.001 ***
9  benefits         covid vaccines - 